# 포지션, 나이, WAR, 잔류 여부 포함 기계 학습 도전

## 테스트 조건 : 투수 야수 통합, 가중치 차이 없이 단순 4년치 성적 합산으로 추정하는 모델 구축

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

df_pi = pd.read_excel('시각화용 투수 데이터.xlsx')
df_ba = pd.read_excel('시각화용 야수 데이터.xlsx')

In [ ]:
# 원본 데이터 보존을 위한 카피
df_pi_m = df_pi.copy()

df_ba_m = df_ba.copy()

In [ ]:
# 일단 필요한 칼럼만 따오기
df_pi_m0 = df_pi_m[['구단명', '선수명', '포지션', 'Age', '종합 WAR', 'FA 계약 연수', 'FA 계약 총액', '잔류 여부']]
df_ba_m0 = df_ba_m[['구단명', '선수명', '포지션', 'Age', 'oWAR', 'dWAR', 'FA 계약 연수', 'FA 계약 총액', '잔류 여부']]

df_ba_m0['종합 WAR'] = df_ba_m0['oWAR'] + df_ba_m0['dWAR']
df_ba_m0 = df_ba_m0.drop(['oWAR', 'dWAR'], axis=1)

df_ba_m0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 544 entries, 0 to 543
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   구단명       544 non-null    object 
 1   선수명       544 non-null    object 
 2   포지션       535 non-null    object 
 3   Age       535 non-null    float64
 4   FA 계약 연수  136 non-null    float64
 5   FA 계약 총액  136 non-null    float64
 6   잔류 여부     136 non-null    object 
 7   종합 WAR    534 non-null    float64
dtypes: float64(4), object(4)
memory usage: 34.1+ KB


In [ ]:
# fa 선언 시점 당시 나이 추출
df_pi_m0['FA 선언 시점 연령'] = df_pi_m0['Age'].iloc[3::4]

# 선수별로 war 덧셈을 진행하기 위해 결측치를 0으로 보충
df_pi_m0['FA 계약 연수'] = df_pi_m0['FA 계약 연수'].fillna(0)
df_pi_m0['FA 계약 총액'] = df_pi_m0['FA 계약 총액'].fillna(0)
df_pi_m0['FA 선언 시점 연령'] = df_pi_m0['FA 선언 시점 연령'].fillna(0)

# 필요 없는 개별 연도 나이 열 탈락
df_pi_m0 = df_pi_m0.drop('Age', axis=1)

# 4줄 단위로 묶어 계산
df_pi_m0['group'] = df_pi_m0.groupby('선수명').cumcount() // 4

df_pi_m0 = df_pi_m0.groupby(['선수명', 'group']).agg({'포지션': lambda x: x.mode().iloc[0],
                                                      '잔류 여부': lambda x: x.mode().iloc[0],
                                                      '종합 WAR': 'sum', 'FA 계약 연수': 'sum',
                                                      'FA 계약 총액' : 'sum', 'FA 선언 시점 연령' : 'sum'}).reset_index()

df_pi_m0

,선수명,group,포지션,잔류 여부,종합 WAR,FA 계약 연수,FA 계약 총액,FA 선언 시점 연령
0,강영식,0,RP,잔류,3.50,4.0,17.0,32.0
1,강윤구,0,RP,은퇴,0.85,0.0,0.0,32.0
2,고효준,0,RP,잔류,1.01,1.0,1.2,36.0
3,권오준,0,RP,잔류,1.52,2.0,6.0,37.0
4,권혁,0,RP,이적,3.91,4.0,32.0,31.0
...,...,...,...,...,...,...,...,...
65,차우찬,1,SP,잔류,10.28,2.0,20.0,33.0
66,채병용,0,RP,잔류,3.67,3.0,10.5,33.0
67,한현희,0,SP,이적,7.58,4.0,40.0,29.0
68,함덕주,0,RP,잔류,4.47,4.0,38.0,28.0


In [ ]:
# 은퇴, 해외 이적 선수 제외
df_pi_m0 = df_pi_m0[df_pi_m0['잔류 여부'].isin(['이적', '잔류'])]
df_pi_m0

,선수명,group,포지션,잔류 여부,종합 WAR,FA 계약 연수,FA 계약 총액,FA 선언 시점 연령
0,강영식,0,RP,잔류,3.50,4.0,17.0,32.0
2,고효준,0,RP,잔류,1.01,1.0,1.2,36.0
3,권오준,0,RP,잔류,1.52,2.0,6.0,37.0
4,권혁,0,RP,이적,3.91,4.0,32.0,31.0
5,금민철,0,RP,잔류,3.83,2.0,7.0,32.0
...,...,...,...,...,...,...,...,...
65,차우찬,1,SP,잔류,10.28,2.0,20.0,33.0
66,채병용,0,RP,잔류,3.67,3.0,10.5,33.0
67,한현희,0,SP,이적,7.58,4.0,40.0,29.0
68,함덕주,0,RP,잔류,4.47,4.0,38.0,28.0


In [ ]:
# 사용할 칼럼만 남기고 최종 드롭
df_pi_m0 = df_pi_m0.drop('group', axis=1)

df_pi_m0.info()

<class 'pandas.core.frame.DataFrame'>
Index: 67 entries, 0 to 69
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   선수명          67 non-null     object 
 1   포지션          67 non-null     object 
 2   잔류 여부        67 non-null     object 
 3   종합 WAR       67 non-null     float64
 4   FA 계약 연수     67 non-null     float64
 5   FA 계약 총액     67 non-null     float64
 6   FA 선언 시점 연령  67 non-null     float64
dtypes: float64(4), object(3)
memory usage: 4.2+ KB


In [ ]:
# fa 선언 시점 당시 나이 추출
df_ba_m0['FA 선언 시점 연령'] = df_ba_m0['Age'].iloc[3::4]

# 선수별로 war 덧셈을 진행하기 위해 결측치를 0으로 보충
df_ba_m0['FA 계약 연수'] = df_ba_m0['FA 계약 연수'].fillna(0)
df_ba_m0['FA 계약 총액'] = df_ba_m0['FA 계약 총액'].fillna(0)
df_ba_m0['FA 선언 시점 연령'] = df_ba_m0['FA 선언 시점 연령'].fillna(0)

# 잔류 여부는

# 필요 없는 개별 연도 나이 열 탈락
df_ba_m0 = df_ba_m0.drop('Age', axis=1)

# 4줄 단위로 묶어 계산
df_ba_m0['group'] = df_ba_m0.groupby('선수명').cumcount() // 4

df_ba_m0 = df_ba_m0.groupby(['선수명', 'group']).agg({'포지션': lambda x: x.mode().iloc[0],
                                                      '잔류 여부': lambda x: x.mode().iloc[0],
                                                      '종합 WAR': 'sum', 'FA 계약 연수': 'sum',
                                                      'FA 계약 총액' : 'sum', 'FA 선언 시점 연령' : 'sum'}).reset_index()

df_ba_m0

,선수명,group,포지션,잔류 여부,종합 WAR,FA 계약 연수,FA 계약 총액,FA 선언 시점 연령
0,강민호,0,C,잔류,16.12,4.0,75.0,28.0
1,강민호,1,C,이적,16.68,4.0,80.0,32.0
2,강민호,2,C,잔류,10.02,4.0,36.0,36.0
3,강한울,0,3B,잔류,1.54,2.0,3.0,32.0
4,고영민,0,2B,은퇴,1.87,0.0,0.0,31.0
...,...,...,...,...,...,...,...,...
131,허경민,0,3B,잔류,10.24,7.0,85.0,30.0
132,허도환,0,C,이적,-0.02,2.0,4.0,37.0
133,홍성흔,0,DH,이적,21.54,4.0,31.0,35.0
134,황재균,0,3B,이적,12.76,4.0,88.0,29.0


In [ ]:
# 은퇴, 해외 이적 선수 제외
df_ba_m0 = df_ba_m0[df_ba_m0['잔류 여부'].isin(['이적', '잔류'])]
df_ba_m0

df_ba_m0 = df_ba_m0.drop('group', axis=1)

df_ba_m0.info()

<class 'pandas.core.frame.DataFrame'>
Index: 132 entries, 0 to 135
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   선수명          132 non-null    object 
 1   포지션          132 non-null    object 
 2   잔류 여부        132 non-null    object 
 3   종합 WAR       132 non-null    float64
 4   FA 계약 연수     132 non-null    float64
 5   FA 계약 총액     132 non-null    float64
 6   FA 선언 시점 연령  132 non-null    float64
dtypes: float64(4), object(3)
memory usage: 8.2+ KB


In [ ]:
df_all_m0 = pd.concat([df_pi_m0, df_ba_m0])

In [ ]:
# 테스트 데이터에도 동일한 처리 진행

df_pi_25 = pd.read_excel('2025 KBO 투수 FA.xlsx')
df_ba_25 = pd.read_excel('2025 KBO 야수 FA.xlsx')

df_pi_25 = df_pi_25[['구단명', '선수명', '포지션', 'Age', '종합 WAR', 'FA 계약 연수', 'FA 계약 총액', '잔류 여부']]
df_ba_25 = df_ba_25[['구단명', '선수명', '포지션', 'Age', 'oWAR', 'dWAR', 'FA 계약 연수', 'FA 계약 총액', '잔류 여부']]

df_ba_25['종합 WAR'] = df_ba_25['oWAR'] + df_ba_25['dWAR']
df_ba_25 = df_ba_25.drop(['oWAR', 'dWAR'], axis=1)

In [ ]:
# fa 선언 시점 당시 나이 추출
df_pi_25['FA 선언 시점 연령'] = df_pi_25['Age'].iloc[3::4]
df_ba_25['FA 선언 시점 연령'] = df_ba_25['Age'].iloc[3::4]

# 선수별로 war 덧셈을 진행하기 위해 결측치를 0으로 보충
df_pi_25['FA 계약 연수'] = df_pi_25['FA 계약 연수'].fillna(0)
df_pi_25['FA 계약 총액'] = df_pi_25['FA 계약 총액'].fillna(0)
df_pi_25['FA 선언 시점 연령'] = df_pi_25['FA 선언 시점 연령'].fillna(0)

df_ba_25['FA 계약 연수'] = df_ba_25['FA 계약 연수'].fillna(0)
df_ba_25['FA 계약 총액'] = df_ba_25['FA 계약 총액'].fillna(0)
df_ba_25['FA 선언 시점 연령'] = df_ba_25['FA 선언 시점 연령'].fillna(0)

# 필요 없는 나이 칼럼 제거
df_pi_25 = df_pi_25.drop('Age', axis=1)
df_ba_25 = df_ba_25.drop('Age', axis=1)

# 4줄 단위로 묶어 계산
df_pi_25['group'] = df_pi_25.groupby('선수명').cumcount() // 4
df_ba_25['group'] = df_ba_25.groupby('선수명').cumcount() // 4

df_pi_25 = df_pi_25.groupby(['선수명', 'group']).agg({'포지션': lambda x: x.mode().iloc[0],
                                                      '잔류 여부': lambda x: x.mode().iloc[0],
                                                      '종합 WAR': 'sum', 'FA 계약 연수': 'sum',
                                                      'FA 계약 총액' : 'sum', 'FA 선언 시점 연령' : 'sum'}).reset_index()

df_ba_25 = df_ba_25.groupby(['선수명', 'group']).agg({'포지션': lambda x: x.mode().iloc[0],
                                                      '잔류 여부': lambda x: x.mode().iloc[0],
                                                      '종합 WAR': 'sum', 'FA 계약 연수': 'sum',
                                                      'FA 계약 총액' : 'sum', 'FA 선언 시점 연령' : 'sum'}).reset_index()

df_pi_25 = df_pi_25.drop('group', axis=1)
df_ba_25 = df_ba_25.drop('group', axis=1)


In [ ]:
df_pi_25 = df_pi_25[df_pi_25['잔류 여부'].isin(['이적', '잔류'])]
df_ba_25 = df_ba_25[df_ba_25['잔류 여부'].isin(['이적', '잔류'])]

df_all_m1 = pd.concat([df_pi_25, df_ba_25])
df_all_m1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19 entries, 0 to 7
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   선수명          19 non-null     object 
 1   포지션          19 non-null     object 
 2   잔류 여부        19 non-null     object 
 3   종합 WAR       19 non-null     float64
 4   FA 계약 연수     19 non-null     float64
 5   FA 계약 총액     19 non-null     float64
 6   FA 선언 시점 연령  19 non-null     float64
dtypes: float64(4), object(3)
memory usage: 1.2+ KB


In [ ]:
# 칼럼 순서를 재정렬
df_all_m0 = df_all_m0[['선수명', '포지션', '종합 WAR', 'FA 선언 시점 연령', '잔류 여부', 'FA 계약 연수', 'FA 계약 총액']]
df_all_m1 = df_all_m1[['선수명', '포지션', '종합 WAR', 'FA 선언 시점 연령', '잔류 여부', 'FA 계약 연수', 'FA 계약 총액']]

df_all_m0.info()

<class 'pandas.core.frame.DataFrame'>
Index: 199 entries, 0 to 135
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   선수명          199 non-null    object 
 1   포지션          199 non-null    object 
 2   종합 WAR       199 non-null    float64
 3   FA 선언 시점 연령  199 non-null    float64
 4   잔류 여부        199 non-null    object 
 5   FA 계약 연수     199 non-null    float64
 6   FA 계약 총액     199 non-null    float64
dtypes: float64(4), object(3)
memory usage: 12.4+ KB


In [ ]:
# 선수명은 계약 결과와 무관하므로 따로 임시 보관
names_0 = df_all_m0['선수명']
names_1 = df_all_m1['선수명']

df_all_m0 = df_all_m0.drop('선수명', axis=1)
df_all_m1 = df_all_m1.drop('선수명', axis=1)

In [ ]:
# 원핫 인코딩으로 포지션과 잔류 여부 인코딩
# 문자형 칼럼인 포지션과 잔류 여부만 골라 인코딩
columns = df_all_m0.select_dtypes(include='object').columns

data = pd.concat([df_all_m0, df_all_m1])
data_oh = pd.get_dummies(data)

df_all_m0 = data_oh.iloc[:len(df_all_m0)]
df_all_m1 = data_oh.iloc[len(df_all_m0):]

df_all_m0.shape, df_all_m1.shape

((199, 18), (19, 18))

In [ ]:
# 훈련용-검증용 데이터 분할(2013~2023)
from sklearn.model_selection import train_test_split

target_m0 = df_all_m0[['FA 계약 연수', 'FA 계약 총액']]
target_m1 = df_all_m1[['FA 계약 연수', 'FA 계약 총액']]

df_all_m0 = df_all_m0.drop(['FA 계약 연수', 'FA 계약 총액'], axis=1)
df_all_m1 = df_all_m1.drop(['FA 계약 연수', 'FA 계약 총액'], axis=1)

x_train, x_test, y_train, y_test = train_test_split(df_all_m0, target_m0, test_size=0.2, random_state=0)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((159, 16), (40, 16), (159, 2), (40, 2))

In [ ]:
# 앞서 종속변수인 계약 연수와 계약 금액과는 선형적인 상관관계가 존재함을 확인한 바 있음
# 단순한 회귀모델은 개별 종속변수에 독립적인 회귀분석을 진행하므로 결과가 의도했던 바와 달라질 위험이 큼
# 요소 사이의 관계를 고려할 수 있는 딥러닝 기법 사용

import tensorflow as tf
from tensorflow.keras import *

# 입력 차원은 칼럼 수, 출력 차원은 종속변수 개수
input_dim = x_train.shape[1]
model = models.Sequential([
    layers.Input(shape=(input_dim, )),
    layers.Dense(64, activation='relu'),
    layers.BatchNormalization(),
    layers.Dense(32, activation='leaky_relu'),
    layers.Dense(2)
])

model.compile(optimizer='adam', loss='mse')
model.fit(x_train, y_train, epochs=50, batch_size=16, validation_data=(x_test, y_test))
model.evaluate(df_all_m1, target_m1)

Epoch 1/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 1118.8447 - val_loss: 1227.7971
Epoch 2/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 1280.6399 - val_loss: 1163.4714
Epoch 3/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 1057.4365 - val_loss: 1121.8938
Epoch 4/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 1005.6718 - val_loss: 1087.0511
Epoch 5/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 1018.5007 - val_loss: 1056.9229
Epoch 6/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 1043.8085 - val_loss: 1019.8060
Epoch 7/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 764.5872 - val_loss: 958.1627
Epoch 8/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 816.9358 - val_loss: 900.5554
Epoch 9/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 977.1360 - val_loss: 854.8158
Epoch 10/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 639.0910 - val_loss: 791.9889
Epoch 11/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 695.1574 - val_loss: 741.0197
Epoch 12/50
10/

71.71129608154297

In [ ]:
y_pred_m1 = model.predict(df_all_m1)
y_pred_m1

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step


array([[ 2.9800744, 19.110516 ],
       [ 2.2596173, 15.601125 ],
       [ 4.3664956, 44.00453  ],
       [ 2.2029915, 12.646718 ],
       [ 5.021601 , 78.72333  ],
       [ 1.184897 ,  5.226613 ],
       [ 2.868823 , 18.414482 ],
       [ 4.2999063, 45.686817 ],
       [ 2.1359215, 11.5786915],
       [ 4.4778905, 49.888927 ],
       [ 5.1311793, 68.44614  ],
       [ 2.5377452, 11.666975 ],
       [ 1.2842451,  5.235661 ],
       [ 3.8064048, 32.285084 ],
       [ 2.0148706, 12.244058 ],
       [ 4.0341606, 32.548634 ],
       [ 3.8807814, 84.23915  ],
       [ 3.5694125, 20.153187 ],
       [ 3.5352309, 41.09872  ]], dtype=float32)

In [ ]:
comparison = pd.DataFrame({
    '선수명' : names_1,
    '예측 연수': y_pred_m1[:, 0],
    '예측 총액': y_pred_m1[:, 1],
    '실제 연수': target_m1['FA 계약 연수'].values,
    '실제 총액': target_m1['FA 계약 총액'].values
})

comparison

,선수명,예측 연수,예측 총액,실제 연수,실제 총액
0,구승민,2.980074,19.110516,4.0,21.0
1,김강률,2.259617,15.601125,4.0,14.0
2,김원중,4.366496,44.004532,4.0,54.0
3,노경은,2.202991,12.646718,3.0,25.0
5,엄상백,5.021601,78.723328,4.0,78.0
6,우규민,1.184897,5.226613,2.0,7.0
7,이용찬,2.868823,18.414482,3.0,10.0
8,임기영,4.299906,45.686817,3.0,15.0
9,임정호,2.135921,11.578691,3.0,12.0
10,장현식,4.477890,49.888927,4.0,52.0


In [ ]:
from sklearn.metrics import r2_score

r2_y1 = r2_score(target_m1.iloc[:, 0], y_pred_m1[:, 0])  # 계약 연수
r2_y2 = r2_score(target_m1.iloc[:, 1], y_pred_m1[:, 1])  # 계약 총액

print(f"연수 R²: {r2_y1:.4f}")
print(f"총액 R²: {r2_y2:.4f}")

# 나이, WAR, 포지션, 잔류 여부만 놓고 봐도 계약 총액을 대체로 근접하게 예측하는 것이 가능
# 금액과 큰 관계가 없었던 포지션을 빼고 다시 분석 진행 예정

연수 R²: -0.1544
총액 R²: 0.8309


with adam
- 렐루 64 x 렐루 64 : (-0.1604, 0.7579)
- 렐루 64 x 리키렐루 64 : (-0.0012, 0.7708)
- 렐루 64 x 리키렐루 32 : (-0.1889, 0.7748)

& batchnormalization
- 렐루 64 x 리키렐루 32 : (-0.1544, 0.8309)


?? 왜 연수 적중률이 이 모양인가 ??

→ KBO가 룰적으로 4년마다 FA 재자격이 부여되므로 4년 계약의 빈도가 다른 연수에 비해 높은 편. 그리고 총액과 달리 연수는 예측치와 실제 값이 1만 달라져도 비율상으로는 30%, 50%의 차이가 날 수 있기 때문에 MSE가 데이터 값에 비해 비대해지는 구조이므로 R^2을 잘 뽑기가 어렵다.

?? 그러면 어떻게 해야 하나 ??

→ 평균을 내면 3.5년 같이 소수점으로 나올 수가 있지만 현실에서 그런 계약은 나오지 않는다. 연수를 아예 범주형 변수처럼 보고 회귀가 아닌 분류로 적용하면 어떨까?

→ 모델 분리 고려